<a href="https://www.ired.info/"> <img src="file/ired_logo.png" alt="ired" style="width: 150px;"/> </a>

# Object Classification- Test Result 物件分類 - 測試訓練結果

In this notebook you will use trained model to detect whether hand sanitizer shuold be pressed.

### Load the trained model 預載已訓練模型

Initialize the PyTorch model.

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Next, load the trained weights from the ``best_model_resnet18.pth`` file that you just trained

In [ ]:
model.load_state_dict(torch.load('ired_model.pth'))

Same as training, you will transfer the job to the GPU.

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Create the preprocessing function 創建預處理函數

Your have loaded your model, but you cannot use it yet.  The format of trained model does not completely match the format of the camera.  
<bg> You need to do some preprocessing. The following steps will be done

1. Convert from HWC layout to CHW layout
2. Scale value of [0, 1] range to [0, 255] range used by camera
3. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

### Display camera and create slider 顯示鏡頭及創建滑塊

You have defined pre-processing function which convert images from the camera to the neural network input format.

Now, try to display your camera with a slider that will display the probability that hand sanitizer should be pressed.
<br> The slider does not display detection result yet.

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
press_slider = widgets.FloatSlider(description='press', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, press_slider]))

### Update slider to show detection result  在滑塊顥示偵測結果

Next, you will create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Display the camera value to slider

In [ ]:
import torch.nn.functional as F
import time
old_time=time.time()

def update(change):
    global press_slider, old_time, prob
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_press = float(y.flatten()[0])
    press_slider.value = prob_press
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Now, you could check if your slider has a higher value when hand-sanitizer should be pressed, a lower value when should not be pressed.
<br>If it behaves very different from what you expected, some problems probably happen at data collection stage.
<br>Check the image in dataset of ``press`` and ``stop`` and ensure images are correct and clear.

You need to mark down a threshold value, which hand-sanitizer will be pressed if slider displayed value is larget than (>) threshold value.

For example, if you observe the slider is around 0.92 to 0.98 probability when you place a hand near it.
You could set your threshold value to 0.9
<br> The demo model has a thershold value of 0.5



### Put Your own thershold value 填寫你的模型閾值: 0.5

Awesome! You have trained a proper model to determine when hand-sanitizer should be pressed.
Now, you will move on the motor part to control the action of pressing hand-sanitizer.

### Stop camera 關閉鏡頭
Properly stop the camera.

In [ ]:
camera.stop()

--End--